In [1]:
from birdcall.data import *
from birdcall.metrics import *
from birdcall.ops import *

import torch
import torchvision
from torch import nn
import numpy as np
import pandas as pd
from pathlib import Path
import soundfile as sf

In [2]:
BS = 16
MAX_LR = 1e-3

In [3]:
classes = pd.read_pickle('data/classes.pkl')

In [4]:
# all_train_items = pd.read_pickle('data/all_train_items.pkl')

# all_train_items_npy = []

# for ebird_code, path, duration in all_train_items:
#     fn = path.stem
#     new_path = Path(f'data/npy/train_resampled/{ebird_code}/{fn}.npy')
#     all_train_items_npy.append((ebird_code, new_path, duration))
    
# pd.to_pickle(all_train_items_npy, 'data/all_train_items_npy.pkl')

In [5]:
splits = pd.read_pickle('data/all_splits.pkl')
all_train_items = pd.read_pickle('data/all_train_items_npy.pkl')

train_items = np.array(all_train_items)[splits[0][0]].tolist()
val_items = np.array(all_train_items)[splits[0][1]].tolist()

In [6]:
# from collections import defaultdict

# class2train_items = defaultdict(list)

# for cls_name, path, duration in train_items:
#     class2train_items[cls_name].append((path, duration))

In [7]:
# pd.to_pickle(class2train_items, 'data/class2train_items.pkl')

In [8]:
class2train_items = pd.read_pickle('data/class2train_items.pkl')

In [9]:
train_ds = MelspecPoolDataset(class2train_items, classes, len_mult=50, normalize=False)
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=BS, num_workers=NUM_WORKERS, pin_memory=True, shuffle=True)

In [10]:
val_items = [(classes.index(item[0]), item[1], item[2]) for item in val_items]
val_items_binned = bin_items_negative_class(val_items)

In [11]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(*list(torchvision.models.resnet34(True).children())[:-2])
        self.classifier = nn.Sequential(*[
            nn.Linear(512, 512), nn.ReLU(), nn.Dropout(p=0.5), nn.BatchNorm1d(512),
            nn.Linear(512, 512), nn.ReLU(), nn.Dropout(p=0.5), nn.BatchNorm1d(512),
            nn.Linear(512, len(classes))
        ])
    
    def forward(self, x):
        x = torch.log10(1 + x)
        mins = x.view(x.shape[0], x.shape[1], -1).min(-1)[0]
        maxs = x.view(x.shape[0], x.shape[1], -1).max(-1)[0]

        good_ones = (maxs - mins) >  1e-4

        x[good_ones] -= x[good_ones].mean((1,2,3))[:, None, None, None]
        x[good_ones] -= x[good_ones].std((1,2,3))[:, None, None, None]

        x[~good_ones] = x[~good_ones].zero_()
        
        bs, im_num = x.shape[:2]
        x = x.view(-1, x.shape[2], x.shape[3], x.shape[4])
        x = self.cnn(x)
        x = x.mean((2,3))
        x = self.classifier(x)
        x = x.view(bs, im_num, -1)
        x = lme_pool(x)
        return x

In [12]:
model = Model().cuda()

In [13]:
for batch in train_dl: break

In [14]:
batch[0].shape

torch.Size([16, 10, 3, 80, 212])

In [15]:
x = model(batch[0].cuda())

In [16]:
import torch.optim as optim
from sklearn.metrics import accuracy_score, f1_score
import time

In [17]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), MAX_LR)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, 5)

In [18]:
# sc_items = pd.read_pickle('data/soundscape_items.pkl')

# sc_items_npy = []
# for labels, path, offset in sc_items:
#     sc_items_npy.append((labels, Path(f'data/npy/shifted/{path.stem}.npy'), offset))
    
# pd.to_pickle(sc_items_npy, 'data/soundscape_items_npy.pkl')

In [19]:
sc_ds = SoundscapeMelspecPoolDataset(pd.read_pickle('data/soundscape_items_npy.pkl'), classes)
sc_dl = torch.utils.data.DataLoader(sc_ds, batch_size=2*BS, num_workers=NUM_WORKERS, pin_memory=True)

In [20]:
t0 = time.time()
for epoch in range(180):
    running_loss = 0.0
    for i, data in enumerate(train_dl, 0):
        model.train()
        inputs, labels = data[0].cuda(), data[1].cuda()
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        if np.isnan(loss.item()): 
            raise Exception(f'!!! nan encountered in loss !!! epoch: {epoch}\n')
        loss.backward()
        optimizer.step()
        scheduler.step()

        running_loss += loss.item()


    if epoch % 5 == 4:
        model.eval();
        preds = []
        targs = []

        for num_specs in val_items_binned.keys():
            valid_ds = MelspecShortishValidatioDataset(val_items_binned[num_specs], classes)
            valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=2*BS, num_workers=NUM_WORKERS, pin_memory=True)

            with torch.no_grad():
                for data in valid_dl:
                    inputs, labels = data[0].cuda(), data[1].cuda()
                    outputs = model(inputs)
                    preds.append(outputs.cpu().detach())
                    targs.append(labels.cpu().detach())

        preds = torch.cat(preds)
        targs = torch.cat(targs)

        f1s = []
        ts = []
        for t in np.linspace(0.4, 1, 61):
            f1s.append(f1_score(preds.sigmoid() > t, targs, average='micro'))
            ts.append(t)
        
        sc_preds = []
        sc_targs = []
        with torch.no_grad():
            for data in sc_dl:
                inputs, labels = data[0].cuda(), data[1].cuda()
                outputs = model(inputs)
                sc_preds.append(outputs.cpu().detach())
                sc_targs.append(labels.cpu().detach())

        sc_preds = torch.cat(sc_preds)
        sc_targs = torch.cat(sc_targs)
        sc_f1 = f1_score(sc_preds.sigmoid() > 0.5, sc_targs, average='micro')
        
        sc_f1s = []
        sc_ts = []
        for t in np.linspace(0.4, 1, 61):
            sc_f1s.append(f1_score(sc_preds.sigmoid() > t, sc_targs, average='micro'))
            sc_ts.append(t)
        
        print(f'[{epoch + 1}, {(time.time() - t0)/60:.1f}] loss: {running_loss / (len(train_dl)-1):.3f}, f1: {max(f1s):.3f}, sc_f1: {max(sc_f1s):.3f}')
        running_loss = 0.0

        torch.save(model.state_dict(), f'models/{epoch+1}_lmepool_simple_log_zero_center_std_norm_{round(max(f1s), 2)}.pth')

[5, 16.0] loss: 0.023, f1: 0.001, sc_f1: 0.000
[10, 31.3] loss: 0.020, f1: 0.008, sc_f1: 0.000
[15, 46.6] loss: 0.018, f1: 0.085, sc_f1: 0.000
[20, 62.1] loss: 0.015, f1: 0.227, sc_f1: 0.000
[25, 77.4] loss: 0.013, f1: 0.353, sc_f1: 0.000
[30, 92.7] loss: 0.012, f1: 0.476, sc_f1: 0.000
[35, 108.0] loss: 0.010, f1: 0.554, sc_f1: 0.000
[40, 123.3] loss: 0.008, f1: 0.593, sc_f1: 0.000
[45, 138.6] loss: 0.007, f1: 0.629, sc_f1: 0.000
[50, 153.8] loss: 0.007, f1: 0.645, sc_f1: 0.000
[55, 169.1] loss: 0.006, f1: 0.645, sc_f1: 0.000
[60, 184.5] loss: 0.005, f1: 0.683, sc_f1: 0.000
[65, 199.7] loss: 0.005, f1: 0.682, sc_f1: 0.000
[70, 215.1] loss: 0.004, f1: 0.692, sc_f1: 0.000
[75, 230.4] loss: 0.004, f1: 0.696, sc_f1: 0.000
[80, 245.7] loss: 0.004, f1: 0.688, sc_f1: 0.000
[85, 261.0] loss: 0.003, f1: 0.700, sc_f1: 0.000
[90, 276.3] loss: 0.003, f1: 0.703, sc_f1: 0.022
[95, 291.6] loss: 0.003, f1: 0.708, sc_f1: 0.000
[100, 306.8] loss: 0.003, f1: 0.705, sc_f1: 0.000
[105, 322.1] loss: 0.003, 

In [24]:
model.load_state_dict(torch.load('models/180_lmepool_simple_log_zero_center_std_norm_0.72.pth'))

<All keys matched successfully>

In [ ]:
t0 = time.time()
for epoch in range(180, 360):
    running_loss = 0.0
    for i, data in enumerate(train_dl, 0):
        model.train()
        inputs, labels = data[0].cuda(), data[1].cuda()
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        if np.isnan(loss.item()): 
            raise Exception(f'!!! nan encountered in loss !!! epoch: {epoch}\n')
        loss.backward()
        optimizer.step()
        scheduler.step()

        running_loss += loss.item()


    if epoch % 5 == 4:
        model.eval();
        preds = []
        targs = []

        for num_specs in val_items_binned.keys():
            valid_ds = MelspecShortishValidatioDataset(val_items_binned[num_specs], classes)
            valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=2*BS, num_workers=NUM_WORKERS, pin_memory=True)

            with torch.no_grad():
                for data in valid_dl:
                    inputs, labels = data[0].cuda(), data[1].cuda()
                    outputs = model(inputs)
                    preds.append(outputs.cpu().detach())
                    targs.append(labels.cpu().detach())

        preds = torch.cat(preds)
        targs = torch.cat(targs)

        f1s = []
        ts = []
        for t in np.linspace(0.4, 1, 61):
            f1s.append(f1_score(preds.sigmoid() > t, targs, average='micro'))
            ts.append(t)
        
        sc_preds = []
        sc_targs = []
        with torch.no_grad():
            for data in sc_dl:
                inputs, labels = data[0].cuda(), data[1].cuda()
                outputs = model(inputs)
                sc_preds.append(outputs.cpu().detach())
                sc_targs.append(labels.cpu().detach())

        sc_preds = torch.cat(sc_preds)
        sc_targs = torch.cat(sc_targs)
        sc_f1 = f1_score(sc_preds.sigmoid() > 0.5, sc_targs, average='micro')
        
        sc_f1s = []
        sc_ts = []
        for t in np.linspace(0.4, 1, 61):
            sc_f1s.append(f1_score(sc_preds.sigmoid() > t, sc_targs, average='micro'))
            sc_ts.append(t)
        
        print(f'[{epoch + 1}, {(time.time() - t0)/60:.1f}] loss: {running_loss / (len(train_dl)-1):.3f}, f1: {max(f1s):.3f}, sc_f1: {max(sc_f1s):.3f}')
        running_loss = 0.0

        torch.save(model.state_dict(), f'models/{epoch+1}_lmepool_simple_log_zero_center_std_norm_{round(max(f1s), 2)}.pth')